In [5]:
import numpy as np

In [3]:
!pip install torch

In [6]:
import pandas as pd

In [7]:
path = '/dgxa_home/se22uari173/big/train.csv'

In [8]:
df = pd.read_csv(path)

In [9]:
df

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [10]:
!pip install spacy

In [11]:
import spacy

/dgxa_home/se22uari173/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
nlp = spacy.load("en_core_web_sm")

/dgxa_home/se22uari173/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/dgxa_home/se22uari173/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [114]:
import torch

In [115]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [116]:
def preprocess_text(text):
    doc = nlp(text)
    cleaned_tokens = [
        token.text for token in doc if not token.is_punct and not token.is_space
    ]
    return " ".join(cleaned_tokens)

In [117]:
import numpy as np
percent = 97
print(np.percentile([len(x) for x in df['article']],percent))

8560.0


In [118]:
print(np.percentile([len(x) for x in df['highlights']],percent))

558.0299999999988


In [119]:
max_input_seq_length = 8660
max_output_seq_length = 563

In [120]:
df = df[:12000]

In [121]:
article_list = list(df['article'])

In [122]:
summary_list = list(df['highlights'])

In [123]:
for i in range(len(article_list)):
    article_list[i] = preprocess_text(article_list[i])
for i in range(len(summary_list)):
    summary_list[i] = preprocess_text(summary_list[i])

In [144]:
maxi = 0
for i in article_list:
    if(len(i)>maxi):
        maxi = len(i)

In [145]:
maxi

14417

In [146]:
def is_validlength(paragraph,max_seq_length):
    return(len(paragraph)<max_input_seq_length)
valid_textsummaries_idx = []
for i in range(len(df['article'])):
    article,summary = article_list[i],summary_list[i]
    if(is_validlength(article,max_input_seq_length) and is_validlength(summary,max_output_seq_length)):
        valid_textsummaries_idx.append(i)

In [147]:
len(valid_textsummaries_idx)

1221

In [148]:
article_list_new = []
summary_list_new = []
for i in valid_textsummaries_idx:
    article_list_new.append(article_list[i])
    summary_list_new.append(summary_list[i])

In [149]:
from torch.utils.data import Dataset,DataLoader

In [150]:
class TextDataset(Dataset):
    def __init__(self,articles,summaries):
        self.articles = articles
        self.summaries = summaries
    def __len__(self):
        return len(self.summaries)
    def __getitem__(self,j):
        return self.articles[j],self.summaries[j]

In [151]:
data = TextDataset(article_list_new,summary_list_new)

In [152]:
data = TextDataset(article_list_new, summary_list_new)
batch_size = 1
train_dataset = DataLoader(data, batch_size=batch_size, shuffle=True)

In [153]:
c = 1
for batch in train_dataset:
    article,summaries = batch
    print(summaries)
    if(c==1):
        break

('Sandor Horvath 65 was attacked by donkeys in Hungary Bitten and trampled on so much looked like he had been attacked by wolves Postmortem revealed the bites and markings had come from the donkeys Pair responsible for killing him are now waiting to be put down',)


In [154]:
!pip install transformers

In [155]:
!pip install sentencepiece

In [156]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [157]:
len(train_dataset)

1221

In [158]:
max = 0
for i in article_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

463


In [159]:
max = 0
for i in summary_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

128


In [160]:
len(batch)

2

In [161]:
input_ids = []
output_ids = []
max_input_seq_length = 1800
max_output_seq_length = 1800
for batch in train_dataset:
    if(len(batch)==2):
        articles, summaries = batch 
        tokenized_articles = tokenizer(list(articles),max_length=max_input_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=False)
        tokenized_summaries = tokenizer(list(summaries),max_length=max_output_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=True)
        input_ids.append(tokenized_articles['input_ids'])
        output_ids.append(tokenized_summaries['input_ids'])

In [162]:
tensor_tokenized_inputs = torch.cat(input_ids,dim=0)

In [163]:
tensor_tokenized_outputs = torch.cat(output_ids,dim=0)

In [164]:
len(input_ids)

1221

In [165]:
import math
from torch import nn
import torch.nn.functional as func

In [166]:
class Multiheadattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,x,mask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batch,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batch,maxseqlength,n))
        return output

In [167]:
class LayerNormalization(torch.nn.Module):
    def __init__(self,eps):
        super(LayerNormalization, self).__init__()
        self.eps = eps 
    def forward(self,inp):
        mean = inp.mean(dim=-1, keepdim=True) 
        var = ((inp - mean) ** 2).mean(dim=-1, keepdim=True)
        std = torch.sqrt(var + self.eps)
        y = (inp - mean) / std
        return y

In [168]:
class feedfowardlayer(nn.Module):
    def __init__(self,weights_dims,hidden_dim,drop_prob):
        super(feedfowardlayer, self).__init__()
        self.weights_dims = weights_dims
        self.hidden_dims = hidden_dim
        self.linearlayer1 = nn.Linear(self.weights_dims,self.hidden_dims)
        self.relu = nn.ReLU()
        self.linearlayer2 = nn.Linear(self.hidden_dims,self.weights_dims)
        self.drop_probability = drop_prob
        self.drop_neurons = nn.Dropout(p=self.drop_probability)
    def forward(self,x):
        out = self.linearlayer1(x)
        out = self.relu(out)
        out = self.drop_neurons(out)
        out = self.linearlayer2(out)
        return out

In [169]:
class Subencoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subencoder,self).__init__()
        self.attention = Multiheadattention(weight_dim,n_heads)
        self.norm = LayerNormalization(1e-6)
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm2 = LayerNormalization(1e-6)
        self.dropout2 = nn.Dropout(p = drop_probability)
    def forward(self,x,mask):
        residual = x
        x = self.attention(x,mask)
        x = self.dropout1(x)
        x = self.norm(x+residual)
        residual = x
        x = self.ffd(x)
        x = self.dropout2(x)
        x = self.norm2(x+residual)
        return x

In [170]:
class Sequentialencoder(nn.Sequential):
    def foward(self,x,mask):
        self.x = x
        self.mask = mask
        for module in self._modules.values():
            x = module(self.x,self.mask)
        return y

In [171]:
class Encoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,layers,drop_probability):
        super(Encoder, self).__init__()
        encoded_layers = []
        for k in range(layers): 
            l = Subencoder(weight_dim, ff_hidden,n_heads,drop_probability)
            encoded_layers.append(l)
        self.layers = nn.ModuleList(encoded_layers)
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return x

In [172]:
class Multiheadcrossattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.kv_matrices = nn.Linear(weights_dim,2*weights_dim)
        self.q_matrix = nn.Linear(weights_dim,weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,y,x,mask):
        batches,maxseqlength,weight_dim = x.size()
        kv = self.kv_matrices(x)
        q = self.q_matrix(y)
        kv = kv.reshape(batches,maxseqlength,self.n_heads,2*self.head_dimensions)
        kv = kv.permute(0,2,1,3)
        k,v = kv.chunk(2,dim=-1)
        q = q.reshape(batches,maxseqlength,self.n_heads,self.head_dimensions)
        q = q.permute(0,2,1,3)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batches,maxseqlength,n))
        return output

In [173]:
class Maskedattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,x,mask,padmask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batch,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask+padmask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batch,maxseqlength,n))
        return output

In [174]:
class Subdecoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subdecoder,self).__init__()
        self.maskedattention = Maskedattention(weight_dim,n_heads)
        self.norm = LayerNormalization(1e-6)
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.Multicrossattention = Multiheadcrossattention(weight_dim,n_heads)
        self.norm2 = LayerNormalization(1e-6)
        self.dropout2 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm3 = LayerNormalization(1e-6)
        self.dropout3 = nn.Dropout(p = drop_probability)
    def forward(self,x,y,mask,padmask,crossmask):
        residual = y
        y = self.maskedattention(y,mask,padmask)
        y = self.dropout1(y)
        y = self.norm(y+residual)
        residual = y
        y = self.Multicrossattention(y,x,crossmask)
        y = self.dropout2(y)
        y = self.norm2(y+residual)
        residual = y
        y = self.ffd(y)
        y = self.dropout3(y)
        y = self.norm3(y+residual)
        return y

In [175]:
class Sequentialdecoder(nn.Sequential):
    def forward(self,x,y,mask,padmask,crossattentpadmask):
        for module in self._modules.values():
            y = module(x,y,mask,padmask,crossattentpadmask)
        return y

In [176]:
class Decoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,layers,drop_probability):
        super().__init__()
        decoded_layers = []
        for k in range(layers):
            l = Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)
            decoded_layers.append(l)
        self.layers = Sequentialdecoder(*[Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)])
    def forward(self,x,y,mask,padmask,crossattentpadmask):
        resdecoder = self.layers(x,y,mask,padmask,crossattentpadmask)
        return resdecoder

In [177]:
vocab_size = tokenizer.vocab_size

In [178]:
vocab_size

32000

In [179]:
class PositionalEncodingWithFeatureTransformation(nn.Module):
    def __init__(self, input_dim, output_dim, max_sequence_length):
        super(PositionalEncodingWithFeatureTransformation, self).__init__()
        self.max_sequence_length = max_sequence_length
        self.output_dim = output_dim
        self.linear = nn.Linear(input_dim, output_dim)
        self.positional_embeddings = nn.Parameter(torch.zeros(max_sequence_length, output_dim), requires_grad=True)
        self.create_positional_encodings()
    def create_positional_encodings(self):
        even_i = torch.arange(0, self.output_dim, 2).float()
        denominator = torch.pow(10000, even_i / self.output_dim)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        self.positional_embeddings.data = torch.flatten(stacked, start_dim=1, end_dim=2)
    def forward(self, x):
        x = self.linear(x)  
        positional_encoded = self.positional_embeddings[:x.size(1), :].unsqueeze(0) 
        return x + positional_encoded

In [180]:
def to_one_hot(token_ids, num_classes):
    one_hot = torch.zeros(token_ids.size(0), token_ids.size(1), num_classes, dtype=torch.float32)
    one_hot.scatter_(2, token_ids.unsqueeze(2), 1) 
    return one_hot

In [181]:
def create_padding_mask(tensor_tokenized_inputs,batch_size,seq_len,n_heads):
    padding_mask = (tensor_tokenized_inputs != 0).float()
    padding_mask = padding_mask.unsqueeze(0)
    mask_2d = padding_mask.unsqueeze(1).expand(-1, n_heads, seq_len, seq_len)  
    padding_mask_expanded = (1.0 - mask_2d) * -3.4028e+38  
    padding_mask_expanded[mask_2d == 1.0] = 0  
    return padding_mask_expanded

In [182]:
def create_padding_mask_2(tensor_tokenized_outputs_, seq_len, n_heads):
    padding_mask = (tensor_tokenized_outputs_ == 0).float()
    repeated_mask_350 = padding_mask.repeat(seq_len, 1)
    repeated_mask_8 = repeated_mask_350.unsqueeze(0).unsqueeze(0).repeat(1, n_heads, 1, 1)  
    padding_mask_expanded = repeated_mask_8 * -3.4028e+38  
    padding_mask_expanded[padding_mask_expanded == 0] = 0  
    return padding_mask_expanded

In [183]:
def create_lookahead_mask(seq_len,n_heads):
    lookahead_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)  
    lookahead_mask = lookahead_mask * -3.4028e+38  
    lookahead_mask = lookahead_mask + torch.tril(torch.zeros(seq_len, seq_len))
    lookahead_mask = lookahead_mask.unsqueeze(0).unsqueeze(0) 
    lookahead_mask_expanded = lookahead_mask.repeat(1, n_heads, 1, 1)  
    return lookahead_mask_expanded

In [184]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, weight_dim, ff_hidden_dim, n_heads, num_layers, max_input_seq_length,max_output_seq_length):
        super(Transformer, self).__init__()
        num_layers = int(num_layers) 
        self.embedding = nn.Embedding(vocab_size, weight_dim)
        self.positional_input_encoding = PositionalEncodingWithFeatureTransformation(32000,512,max_input_seq_length)
        self.positional_output_encoding = PositionalEncodingWithFeatureTransformation(32000,512,max_output_seq_length)
        self.encoder = Encoder(weight_dim, ff_hidden_dim, n_heads, num_layers, 0.1)
        self.decoder = Decoder(weight_dim, ff_hidden_dim, n_heads, num_layers, 0.1)
        self.fc_out = nn.Linear(weight_dim, vocab_size)
    def forward(self,src,tgt,src_mask,tgt_mask,lkahd_mask,crossattention_mask):
        enc_out = self.encoder(src,src_mask)
        dec_out = self.decoder(tgt,enc_out,lkahd_mask,tgt_mask,crossattention_mask)
        return self.fc_out(dec_out)

In [185]:
weight_dim = 512
ff_hidden_dim = 2048
n_heads = 8
num_layers = 6
vocab_size = tokenizer.vocab_size
max_seq_input_length = 1800
max_seq_output_length = 1800
transformer = Transformer(vocab_size,weight_dim, ff_hidden_dim, n_heads, num_layers,max_seq_input_length,max_seq_output_length)

In [186]:
transformer.to(device)

Transformer(
  (embedding): Embedding(32000, 512)
  (positional_input_encoding): PositionalEncodingWithFeatureTransformation(
    (linear): Linear(in_features=32000, out_features=512, bias=True)
  )
  (positional_output_encoding): PositionalEncodingWithFeatureTransformation(
    (linear): Linear(in_features=32000, out_features=512, bias=True)
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x Subencoder(
        (attention): Multiheadattention(
          (qkv_matrices): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffd): feedfowardlayer(
          (linearlayer1): Linear(in_features=512, out_features=2048, bias=True)
          (relu): ReLU()
          (linearlayer2): Linear(in_features=2048, out_features=512, bias=True)
          (drop_neurons): Dropout(p=0.1, inplace=False)
    

In [187]:
class Word2VecEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
    def forward(self, token_ids):
        return self.embedding(token_ids)

In [188]:
embedding_matrix_inputs = nn.Linear(512,512)
embedding_matrix_outputs = nn.Linear(512,512)
positional_embeddings = PositionalEncodingWithFeatureTransformation(input_dim=512,output_dim=512,max_sequence_length=1800)
word2vec = Word2VecEmbedding(32000,512)
optimizer = torch.optim.Adam(list(transformer.parameters()) + list(embedding_matrix_inputs.parameters()) + list(embedding_matrix_outputs.parameters()) + list(positional_embeddings.parameters()),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
for i in range(35):
    weights_dim = 512
    ff_hidden = 2048
    n_heads = 8
    drop_probability = 0.1
    num_layers = 3
    batch_size = 1
    for i in range(len(train_dataset)):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        current_batch_inputs = tensor_tokenized_inputs[start_idx:end_idx]  
        current_batch_outputs = tensor_tokenized_outputs[start_idx:end_idx]
    with torch.no_grad(): 
           X = word2vec(current_batch_inputs)
    with torch.no_grad():
           Y = word2vec(current_batch_outputs)
    embedded_X = embedding_matrix_inputs(X)
    embedded_Y = embedding_matrix_outputs(Y)
    positional_embeddingsx = positional_embeddings(X) 
    final_X = embedded_X + positional_embeddingsx
    positional_embeddingsy = positional_embeddings(Y) 
    final_Y = embedded_Y + positional_embeddingsy
    src_mask = create_padding_mask(current_batch_inputs,1,1800,8)
    tgt_mask = create_padding_mask(current_batch_outputs,1,1800,8)
    lkahd_mask = create_lookahead_mask(1800,8)
    crossattention_mask = create_padding_mask_2(current_batch_outputs,1800,8)
    logits = transformer(final_X,final_Y,src_mask,tgt_mask,lkahd_mask,crossattention_mask)
    probabilities = func.softmax(logits,dim=-1)
    predicted_tokens = torch.argmax(probabilities,dim=-1)
    predicted_text = tokenizer.decode(predicted_tokens[0].tolist())
    loss = criterion(logits.view(-1, logits.size(-1)), current_batch_outputs.view(-1))
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(10.5005, grad_fn=<NllLossBackward0>)
tensor(8.1426, grad_fn=<NllLossBackward0>)
tensor(6.5037, grad_fn=<NllLossBackward0>)
tensor(5.0325, grad_fn=<NllLossBackward0>)
tensor(5.0470, grad_fn=<NllLossBackward0>)
tensor(2.9035, grad_fn=<NllLossBackward0>)
tensor(2.1341, grad_fn=<NllLossBackward0>)
tensor(1.1361, grad_fn=<NllLossBackward0>)
tensor(0.6309, grad_fn=<NllLossBackward0>)
tensor(0.4671, grad_fn=<NllLossBackward0>)
tensor(0.3474, grad_fn=<NllLossBackward0>)
tensor(0.2563, grad_fn=<NllLossBackward0>)
tensor(0.2103, grad_fn=<NllLossBackward0>)
tensor(0.1391, grad_fn=<NllLossBackward0>)
tensor(0.1200, grad_fn=<NllLossBackward0>)
tensor(0.0775, grad_fn=<NllLossBackward0>)
tensor(0.0558, grad_fn=<NllLossBackward0>)
tensor(0.0446, grad_fn=<NllLossBackward0>)
tensor(0.0359, grad_fn=<NllLossBackward0>)
tensor(0.0262, grad_fn=<NllLossBackward0>)
tensor(0.0199, grad_fn=<NllLossBackward0>)
tensor(0.0150, grad_fn=<NllLossBackward0>)
tensor(0.0127, grad_fn=<NllLossBackward0>)
tensor(0.0

In [189]:
predicted_text

'Phillipe Mexes dismissed for grabbing the throat of Stefano Mauri AC Milan defender also appeared to clutch neck of Lazio player Lorik Cana Lazio beat AC Milan 3 1 at the Stadio Olimpico on Saturday in Serie A</s> beat the     </s> also too the  ozi the LaOlimp tozi Stadizizi  La Mexezi  Lazi Lao on theo  o Saturdayo Serie Milan A Mexezi of of Stefano Ma A Milan  too Lazi appeared to clutch neck Stadirik Cano      o  La grabbing the Loo beat  ozi La  dismissed to  zizi   Laoo the   Mexezi Laoziograbbing  Milan  to AC Lo Lazio player at  Stadio  dismissed   throatooorik Canoooo beat ozi tooo throatoo o  toe  in beate the Can oodefenderoo  Stadi o     o of o the oo  of playero ofoooo 1 atgrabbingo  o o Cano playeroograbbing ooograbbingo o grabbingo defender oicoo    defender o o Milan defender   ofo defender of Lazi    o  o   o     Milan defender of La  o defender zi  ooo 1 at  rik Can   defender o     throat Milan defender oodefenderoo                                                   

In [190]:
end_pos = predicted_text.find("</s>")
if end_pos != -1:
    truncated_text = predicted_text[:end_pos]
else:
    truncated_text = predicted_text 
cleaned_text = " ".join(truncated_text.split())
print(cleaned_text)

Phillipe Mexes dismissed for grabbing the throat of Stefano Mauri AC Milan defender also appeared to clutch neck of Lazio player Lorik Cana Lazio beat AC Milan 3 1 at the Stadio Olimpico on Saturday in Serie A


In [191]:
!pip install gtts

In [192]:
from gtts import gTTS

In [193]:
import os

In [194]:
!pip install pygame

In [195]:
!pip install pydub

In [196]:
from pydub import AudioSegment
from pydub.playback import play
myobj = gTTS(text=cleaned_text, lang='en', slow=False)
myobj.save("welcome.mp3")
from IPython.display import Audio
Audio("welcome.mp3")